<a href="https://colab.research.google.com/github/ncduy/catboy/blob/main/sell10contract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

api_key = "PK0ZFR34PNBANAPIJCQM"
secret_key = "g3bD2DIu7cxKaSHadhRH5uq3k3Q2OqvUJUNwQAh2"

#### We use paper environment for this example ####
paper = True  # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = None

In [82]:
import os

if api_key is None:
    api_key = os.environ.get('ALPACA_API_KEY')

if secret_key is None:
    secret_key = os.environ.get('ALPACA_SECRET_KEY')

In [83]:
# install alpaca-py if it is not available
try:
    import alpaca
except ImportError:
    !python3 -m pip install alpaca-py
    import alpaca

In [84]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import calendar

from alpaca.trading.client import TradingClient
from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest,
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    MarketOrderRequest,
    OptionLegRequest,
    ReplaceOrderRequest,
    LimitOrderRequest,
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderStatus,
    TimeInForce,
    OrderClass,
    ContractType,
    OrderType
)
from alpaca.common.exceptions import APIError

In [85]:
# setup client
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [86]:
# First, let us see if the account is enabled for multi-leg options trading
acct = trade_client.get_account()
if acct.options_trading_level >= 3:
    print("Account is enabled for multi-leg options trading!")
else:
    print("Account is not enabled for multi-leg options trading!")

Account is enabled for multi-leg options trading!


In [87]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockLatestTradeRequest
# Initialize Stock Data Client
stock_client = StockHistoricalDataClient(api_key, secret_key)

# Create a request to get the latest trade price for TSLA
trade_request = StockLatestTradeRequest(symbol_or_symbols=["TSLA"])
latest_trade = stock_client.get_stock_latest_trade(trade_request)

# Extract TSLA's latest price
tsla_price = latest_trade["TSLA"].price
print(f"Current TSLA Price: ${tsla_price}")

Current TSLA Price: $361.365


In [88]:
from alpaca.data.historical.option import OptionHistoricalDataClient

# Request all available TSLA option contracts
req = GetOptionContractsRequest(
    underlying_symbols=["TSLA"],  # Filter for TSLA options
    status=AssetStatus.ACTIVE,    # Only fetch active contracts
    expiration_date_lte='2025-02-21',
    type=ContractType.CALL,
    limit=1000                     # Increase limit if needed
)

# Fetch contracts
contracts = trade_client.get_option_contracts(req)
contracts

# Filter contracts by strike price
contracts_sorted = sorted(contracts.option_contracts, key=lambda c: abs(c.strike_price - tsla_price))
contracts_above = [c for c in contracts_sorted if c.strike_price > tsla_price][:3]
contracts_below = [c for c in contracts_sorted if c.strike_price < tsla_price][:3]

# Select the final 6 contracts
selected_contracts = contracts_below + contracts_above
selected_contracts

[{   'close_price': '4.36',
     'close_price_date': datetime.date(2025, 2, 18),
     'expiration_date': datetime.date(2025, 2, 21),
     'id': 'e280c2b6-1f67-40b1-8dd8-c0860a191188',
     'name': 'TSLA Feb 21 2025 360 Call',
     'open_interest': '23168',
     'open_interest_date': datetime.date(2025, 2, 14),
     'root_symbol': 'TSLA',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,
     'strike_price': 360.0,
     'style': <ExerciseStyle.AMERICAN: 'american'>,
     'symbol': 'TSLA250221C00360000',
     'tradable': True,
     'type': <ContractType.CALL: 'call'>,
     'underlying_asset_id': UUID('8ccae427-5dd0-45b3-b5fe-7ba5e422c766'),
     'underlying_symbol': 'TSLA'},
 {   'close_price': '5.3',
     'close_price_date': datetime.date(2025, 2, 18),
     'expiration_date': datetime.date(2025, 2, 21),
     'id': '515a311c-37a1-481e-8c3b-5a3137c3ef91',
     'name': 'TSLA Feb 21 2025 357.5 Call',
     'open_interest': '3720',
     'open_interest_date': datetime.date(202

In [96]:
# Define stock and quantity
stock_symbol = "TSLA"
quantity = 800  # Buy 100 shares

# simple, market order
# you can specify:
# or notional value (e.g. 100 USD) (which is in the next example)
req = MarketOrderRequest(
    symbol = stock_symbol,
    qty = quantity,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
#res = trade_client.submit_order(req)
#res

# Fetch account positions
positions = trade_client.get_all_positions()

# Find TSLA position
tsla_position = next((p for p in positions if p.symbol == "TSLA"), None)

# Check if you own at least 100 shares
if tsla_position and int(tsla_position.qty) >= 100:
    print(f"You own {tsla_position.qty} TSLA shares ✅")
else:
    print("You do not own 100 TSLA shares yet ❌")

You own 1800 TSLA shares ✅


In [97]:

# ✅ Extract symbols and strike prices
strike_prices = {contract.symbol: contract.strike_price for contract in selected_contracts}

# Sort contracts by strike price (ascending order)
sorted_contracts = sorted(strike_prices.items(), key=lambda x: x[1])

# Extract sorted symbols
option_symbols = [symbol for symbol, _ in sorted_contracts]
# ✅ Extract the option symbols correctly
#option_symbols = [contract.symbol for contract in contracts.option_contracts]
#strike_prices = {contract.symbol: contract.strike_price for contract in contracts.option_contracts}
#setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(api_key, secret_key, trade_api_url)


# Split symbols into batches of 100
batch_size = 100
option_quotes = {}

for i in range(0, len(option_symbols), batch_size):
    batch = option_symbols[i:i + batch_size]  # Get a batch of 100 symbols
    req = OptionLatestQuoteRequest(symbol_or_symbols=batch)
    #req = OptionSnapshotRequest(symbol_or_symbols=batch)
    try:
        res = option_historical_data_client.get_option_latest_quote(req)
        #res = option_historical_data_client.get_option_snapshot(req)
        option_quotes.update(res)  # Merge results into main dictionary
    except Exception as e:
        print(f"Error fetching batch {i // batch_size + 1}: {e}")

# Print results
print(f"Retrieved quotes for {len(option_quotes)} option contracts.")
option_quotes

order_legs = []

for symbol, strike_price in sorted_contracts:
    bid_price = option_quotes[symbol].bid_price if symbol in option_quotes else "N/A"
    ask_price = option_quotes[symbol].ask_price if symbol in option_quotes else "N/A"

    print(f"{symbol:<20} {strike_price:<15} {bid_price:<15} {ask_price:<15}")

    # place buy put option order
    # - we can place buy put option order same as buy stock/crypto order
    req = MarketOrderRequest(
        symbol=symbol,
        qty=1,
        side=OrderSide.SELL,
        type=OrderType.LIMIT,  # Limit order type
        time_in_force=TimeInForce.DAY,  # Good Till Canceled
        limit_price=ask_price  # Sell only if price reaches this
    )
    res = trade_client.submit_order(req)
    print(f"Sell Order Submitted: {res}")

Retrieved quotes for 6 option contracts.
TSLA250221C00355000  355.0           9.28            9.43           
Sell Order Submitted: id=UUID('2bc5d124-2346-474e-b181-dbe2ec42cb9d') client_order_id='738bdecd-992b-4533-ac75-fa466634b08c' created_at=datetime.datetime(2025, 2, 19, 16, 9, 48, 962569, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2025, 2, 19, 16, 9, 48, 962621, tzinfo=TzInfo(UTC)) submitted_at=datetime.datetime(2025, 2, 19, 16, 9, 48, 960943, tzinfo=TzInfo(UTC)) filled_at=None expired_at=None expires_at=datetime.datetime(2025, 2, 19, 21, 0, tzinfo=TzInfo(UTC)) canceled_at=None failed_at=None replaced_at=None replaced_by=None replaces=None asset_id=UUID('bfb58d70-247d-4ca2-861e-c7e3d02f624b') symbol='TSLA250221C00355000' asset_class=<AssetClass.US_OPTION: 'us_option'> notional=None qty='1' filled_qty='0' filled_avg_price=None order_class=<OrderClass.SIMPLE: 'simple'> order_type=<OrderType.MARKET: 'market'> type=<OrderType.MARKET: 'market'> side=<OrderSide.SELL: 'sell'> time